In [ ]:
# also called Series to Scalar UDF
from pyspark.sql import SparkSession
from functools import reduce
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
spark = SparkSession.builder.appName("CH 09 - UDF on grouped data").getOrCreate()

In [ ]:
gsod = (
    reduce(
        lambda x, y: x.unionByName(y, allowMissingColumns=True),
        [
            spark.read.parquet(f"/opt/spark/data/gsod_noaa/gsod{year}.parquet")
            for year in range(2019, 2021)
        ],
    )
    .dropna(subset=["year", "mo", "da", "temp"])
    .where(F.col("temp") != 9999.9)
    .drop("date")
)

In [ ]:
@F.pandas_udf(T.DoubleType())
def rate_of_change_temp(day: pd.Series, temp: pd.Series) -> float:
    return LinearRegression().fit(X=day.astype(int).values.reshape(-1, 1), y=temp).coef_[0]

In [ ]:
result = gsod.groupby("stn", "year", "mo").agg(
    rate_of_change_temp(gsod["da"], gsod["temp"]).alias(
        "rt_change_temp"
    )
)

result.show(5, False)

In [ ]:
# the group map pattern: maps over each batch and returns a DataFrame which are later combined into a single DataFrame
# the function must return a complete DataFrame, which means that all of the columns need to be returned - 
# including the ones we groupped by
# the pandas_udf decorator is NOT NEEDED in this case

def scale_temperature(temp_by_day: pd.DataFrame) -> pd.DataFrame:
    temp = temp_by_day.temp
    answer = temp_by_day[["stn", "year", "mo", "da", "temp"]]

    if temp.min() == temp.max():
        return answer.assign(temp_norm=0.5)

    return answer.assign(
        temp_norm=(temp - temp.min()) / (temp.max() - temp.min())
    )

In [ ]:
schema = T.StructType([
    T.StructField("stn", T.StringType()),
    T.StructField("year", T.StringType()),
    T.StructField("mo", T.StringType()),
    T.StructField("da", T.StringType()),
    T.StructField("temp", T.DoubleType()),
    T.StructField("temp_norm", T.DoubleType()),
])

In [ ]:
gsod_map = gsod.groupby("stn", "year", "mo").applyInPandas(
    scale_temperature,
    schema=schema
)

In [ ]:
gsod_map.show(5, False)